For PHYTOMap, probes must be:
40 - 50 nucleotide length.
Padlock:Primer ratio maximum of 22:26.
There must be a 2 nuc gap between the Padlock and Primer

Padlock and Primer must each have:
GC content of 40% - 60% 
Mt ~60C 
They must be unique to each bacteria and Arabidopsis 

Padlock must have unique barcode 
Padlock and Primers must have complementary regions. 

Having multiple probes is good: aim for 4.

In [1]:
#import packages
import pandas as pd #v.2.2.2
import numpy as np #v1.26.4
import matplotlib.pyplot as plt #v.3.9.2
import scipy.stats as stats
from Bio import SeqIO
from Bio.SeqUtils import MeltingTemp as mt
from Bio.SeqUtils import gc_fraction
from Bio.Seq import Seq
import pandas as pd



since Tatsuya wants to make use of the entire genome. I've made BlastDBs of 4 bacteria genomes + Arabidopsis Col-0; (4 BlastDBs each with one bac genome removed, plus one BLASTdb with all of them).
I'm going to take the agrobacterium genome (as it's the smallest) and split it into 52-mers. I'm then going to filter this by GC content. Then, I will blast this locally, using a command-line call. This should give me unique potential probes which I can then split into 25-mers and find the melting temperature of. An outgoing issue is limiting it to constitutively expressed, protein-coding genetic material; good luck charlie!

In [27]:
# my incredibly flawless and scientific method to find a 52-mer with a 50% GC content will have a mt of ~72.

myseq1 = Seq("CGCGCGCGCGCGCGCGCGCGCGCGCATATATATATATATATATATATATATGT")
myseq2 = Seq("AAACGATTTTAGAGATCTAGGTCAGCCCGGCACTGGTCCCAAATTCTGTGCAC")
myseq3 = Seq("CGTTCAACGCAGTTCGACTAAGCGTCATGAGGGAAATTCTACTCTTGCAGTGC")
myseq4 = Seq("CACATCAGCACATCGCGTTAAGGGAAAACAATTGCCCGTTTTATGCGTGCGTC")
myseq5 = Seq("ATATCGGCGCACGGACTTTACACAGTGTACCTGAGTTCGGACCTTGAAATCGT")
myseq6 = Seq("CTAGCCACAATGCGCATGTAAGGAATTGTCACTTTGGGCTCAACACTGACTGC")

mt1 = '%0.2f' % mt.Tm_NN(myseq1,dnac1=250, dnac2=250, Na=50)
mt2 = '%0.2f' % mt.Tm_NN(myseq2,dnac1=250, dnac2=250, Na=50)
mt3 = '%0.2f' % mt.Tm_NN(myseq3,dnac1=250, dnac2=250, Na=50)
mt4 = '%0.2f' % mt.Tm_NN(myseq4,dnac1=250, dnac2=250, Na=50)
mt5 = '%0.2f' % mt.Tm_NN(myseq5,dnac1=250, dnac2=250, Na=50)
mt6 = '%0.2f' % mt.Tm_NN(myseq6,dnac1=250, dnac2=250, Na=50)

print(mt1,mt2,mt3,mt4,mt5,mt6)
print(np.mean([72.53, 72.17, 71.82, 73.01, 72.53, 72.26]))

# and if i split these sequences in half? do we get a mt of hopefully around 60C?
seq_test_list = [myseq1,myseq2,myseq3,myseq4,myseq5,myseq6]

results = []
for seq in seq_test_list:
    mt_26 = [mt.Tm_NN(seq[0:26],dnac1=250, dnac2=250, Na=50),mt.Tm_NN(seq[27:52],dnac1=250, dnac2=250, Na=50)]
    results.append(mt_26)

print(results)

# yeah! in the ball park!


72.53 72.17 71.82 73.01 72.53 72.26
72.38666666666666
[[85.4096983808044, 34.25773290625153], [55.54818027844675, 64.45985080083557], [63.17356530041576, 57.40484923281855], [62.31418441386046, 63.50496372029204], [63.03005494340198, 59.19594653087131], [60.540660830668, 60.778044818247224]]


### define functions 
(I know, they're a bit ugly!)

In [2]:
def possible_probes(input_file_path): #note, this will "append" if file is already present!
    results = []
    all_fasta_list = []
    window_size = 52 #the contig size; as we want 2 25 nt probes with a 2 nt gap.
    for record in SeqIO.parse(input_file_path, "fasta"):
        all_fasta_list.append([record.id,record.seq]) #this is a list of all the genes.
    for gene in all_fasta_list:
        for i in range(len(gene[1]) - window_size + 1):
            temporary_kmer = (gene[1][i: i + window_size])
            try:
                tm = mt.Tm_NN(                          
                    temporary_kmer,
                    dnac1=250, dnac2=250, 
                    Na=50
                )
                gc_content = gc_fraction(
                    temporary_kmer
                )
                if abs(tm - 72.4) <= 5 and 0.4 <= gc_content <= 0.6:
                    results.append((temporary_kmer, i, tm, gc_content,gene[0]))
            except:
                continue
    return results #this outputs a list of lists of the results saved to the fasta files!

# run the blast shell scripts between these functions.

def post_blast_processing(blast_hits_filepath,possible_probes,annotation_filepath):
    #import blast hits and add column names
    blast_hits = pd.read_csv(blast_hits_filepath, names = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"], usecols=["qseqid"])
    #use it to filter our possible_probes
    probes_df = pd.DataFrame(possible_probes, columns = ["probe_seq", "start_position_in_gene", "melting temp", "GC", "geneID" ]) #relabelling
    probes_df['ID'] = probes_df['start_position_in_gene'].astype(str) + '_of_' + probes_df['geneID'].astype(str)
    unique_probes = probes_df.loc[~probes_df['ID'].isin(blast_hits['qseqid'])].reset_index(drop=True)

    #split 52-mer probes into 25-mer probes and recheck melting temperature and gc content

    unique_probes['probe1'] = unique_probes.loc[:,'probe_seq'].str[0:25]
    unique_probes['probe2'] = unique_probes.loc[:,'probe_seq'].str[27:50] # 2 nt gap.
    #recheck melting temp
    probe1_series = pd.Series(unique_probes.loc[:,'probe1'])
    probe2_series = pd.Series(unique_probes.loc[:,'probe2'])
    mt_list1 = []
    mt_list2 = []

    for probe in probe1_series:
        melting_temp = mt.Tm_NN(probe,dnac1=250, dnac2=250, Na=50)
        mt_list1.append(melting_temp)

    for probe in probe2_series:
        melting_temp = mt.Tm_NN(probe,dnac1=250, dnac2=250, Na=50)
        mt_list2.append(melting_temp)

    unique_probes['mt1'] = mt_list1
    unique_probes['mt2'] = mt_list2

    # and gc content of these half probes
    gc_list1 = []
    gc_list2 = []

    for probe in probe1_series:
        gc_content = gc_fraction(probe)
        gc_list1.append(gc_content)

    for probe in probe2_series:
        gc_content = gc_fraction(probe)
        gc_list2.append(gc_content)

    unique_probes['gc1'] = gc_list1
    unique_probes['gc2'] = gc_list2

    #filter to useful probes
    good_probes = unique_probes[
        (abs(unique_probes['mt1'] - 60) <= 2) &
        (abs(unique_probes['mt2'] - 60) <= 2) &
        (0.4 <= unique_probes['gc1']) & (unique_probes['gc1'] <= 0.6) &
        (0.4 <= unique_probes['gc2']) & (unique_probes['gc2'] <= 0.6)           
        ]
    
    #find how many probes per gene and attach KO annotations
    probes_by_geneID = pd.DataFrame(good_probes['geneID'].value_counts())
    probes_by_geneID = probes_by_geneID.rename(columns={"count": "probe_count"})

    #the downloaded annotations are tab-delimited
    KO_df = pd.read_csv(annotation_filepath,sep='\t')
    KO_df['gene_oid'] = KO_df['gene_oid'].astype(str)

    probes_KO = pd.merge(probes_by_geneID, KO_df, left_on='geneID', right_on='gene_oid').iloc[:,[0,1,3,8,10,11]]
    return [good_probes,probes_KO]


### Initial probe screening

In [3]:
baci21_input_path = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/all_just_cds/coding_seq_only_probes/bacillus_2563366514.genes.fna"
baci21_probes = possible_probes(baci21_input_path)

In [5]:
#write to text file output
baci21_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/baci21_possible.fasta", "a")
for maybe_probe in baci21_probes:
    string = f">{maybe_probe[1]}_of_{maybe_probe[4]} \n" + str(maybe_probe[0]) + "\n"
    baci21_fasta.write(string)
baci21_fasta.close()

In [ ]:
## Agrobacteria
agro_input_path = '/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/coding_seq_only/agrobacterium_2561511224.genes.fna'
agro_probes = possible_probes(agro_input_path)

## Bacillus
baci_input_path = '/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/coding_seq_only/bacillus_2517572206.genes.fna'
baci_probes = possible_probes(baci_input_path)

## flavobacteria
flavo_input_path = '/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/coding_seq_only/flavobacterium_2563366720.genes.fna'
flavo_probes = possible_probes(flavo_input_path)

## Leifsonia
leif_input_path = '/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/coding_seq_only/leifsona_2522572063.genes.fna'
leif_probes = possible_probes(leif_input_path)

## Pseud
pseud_input_path = '/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/coding_seq_only/pseud_2228664007.genes.fna'
pseud_probes = possible_probes(pseud_input_path)

## WCS365
wcs365_input_path = '/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/coding_seq_only/WCS365_cds.fna'
wcs365_probes = possible_probes(wcs365_input_path)


#check output
output_list = [agro_probes,baci_probes,flavo_probes,leif_probes,pseud_probes,wcs365_probes]
names_list = ["agro", "baci", "flavo", "leif", "pseud", "wcs365"]
name_tick = 0
print("where: probe_seq | start_position_in_gene | melting temp | GC content | geneID", '\n')
for output in output_list:
    print(
        f"first {names_list[name_tick]} probe:", output[0], '\n',
        f"total {names_list[name_tick]} probes: ",len(output)
    )

#write to text file output
agro_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/agro_possible.fasta", "a")
for maybe_probe in agro_probes:
    string = f">{maybe_probe[1]}_of_{maybe_probe[4]} \n" + str(maybe_probe[0]) + "\n"
    agro_fasta.write(string)
agro_fasta.close()

baci_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/baci_possible.fasta", "a")
for maybe_probe in baci_probes:
    string = f">{maybe_probe[1]}_of_{maybe_probe[4]} \n" + str(maybe_probe[0]) + "\n"
    baci_fasta.write(string)
baci_fasta.close()

flavo_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/flavo_possible.fasta", "a")
for maybe_probe in flavo_probes:
    string = f">{maybe_probe[1]}_of_{maybe_probe[4]} \n" + str(maybe_probe[0]) + "\n"
    flavo_fasta.write(string)
flavo_fasta.close()

leif_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/leif_possible.fasta", "a")
for maybe_probe in leif_probes:
    string = f">{maybe_probe[1]}_of_{maybe_probe[4]} \n" + str(maybe_probe[0]) + "\n"
    leif_fasta.write(string)
leif_fasta.close()

pseud_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/pseud_possible.fasta", "a")
for maybe_probe in pseud_probes:
    string = f">{maybe_probe[1]}_of_{maybe_probe[4]} \n" + str(maybe_probe[0]) + "\n"
    pseud_fasta.write(string)
pseud_fasta.close()

wcs365_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/wcs365_possible.fasta", "a")
for maybe_probe in wcs365_probes:
    string = f">{maybe_probe[1]}_of_{maybe_probe[4]} \n" + str(maybe_probe[0]) + "\n"
    wcs365_fasta.write(string)
wcs365_fasta.close()



where: probe_seq | start_position_in_gene | melting temp | GC content | geneID 

first agro probe: (Seq('CAGAAACCGGCATCGACATGGCTGTTCCGCAGAGCTCAACCCTTGCGACCCA'), 7, 77.25578017378962, 0.5961538461538461, '2562541315') 
 total agro probes:  2129340
first agro probe: (Seq('CCTGCAGTATCTCATGCTGAAAACCAAGAAGCTGTAAAACAACTTACTCCAA'), 0, 68.19014210611846, 0.40384615384615385, '2518120388') 
 total agro probes:  1400565
first agro probe: (Seq('TGTAGGACCACGACATTTTATGCATGCAGAACCGGAAGTTACTGGAAAGTGA'), 0, 70.34675648662079, 0.4423076923076923, '2563780541') 
 total agro probes:  1182763
first agro probe: (Seq('GAAGCCCGGCGACAACCTCAAGTGGGTCGTCAGCTACCAGAACAACACGGGT'), 179, 76.76611293413566, 0.5961538461538461, '2522792368') 
 total agro probes:  198271
first agro probe: (Seq('ATGAGTAGCCTCTTAACAAGCGCCCAATTACAGCTTTTATTTGCGTTGTGCT'), 0, 70.34055691670147, 0.4230769230769231, '2230705794') 
 total agro probes:  2429386
first agro probe: (Seq('ATGAAGTCGGTCGAAGAGCAGCTAGCGCTGATCAAACGTGGTGCGGAAGAAC'), 0, 74.3

### BLAST

so, i've made a shell scripts to make the blast databases and run the blasts, as such.

I could muck about with subprocess to try get this to run in python, but really. just copy and paste the above text, remove most the hashtags, save it to the appropriate directory as a .sh file, make executable with chmod +x ./path/to/file.sh, and run. voila, your blast results!



In [ ]:
# #!/bin/sh
 
# cd ~/Documents/nobori_PHYTOMap_rotation/probes/blast_dbs

# # concatenate the transcriptome databases into the relevant groups

# cat agrobacterium_2561511224.genes.fna bacillus_2517572206.genes.fna flavobacterium_2563366720.genes.fna leifsona_2522572063.genes.fna pseud_2228664007.genes.fna WCS365_cds.fna Col0.fasta > all_blastdb.fasta

# cat bacillus_2517572206.genes.fna flavobacterium_2563366720.genes.fna leifsona_2522572063.genes.fna pseud_2228664007.genes.fna WCS365_cds.fna Col0.fasta > notagro_blastdb.fasta

# cat agrobacterium_2561511224.genes.fna flavobacterium_2563366720.genes.fna leifsona_2522572063.genes.fna pseud_2228664007.genes.fna WCS365_cds.fna Col0.fasta > notbaci_blastdb.fasta

# cat agrobacterium_2561511224.genes.fna bacillus_2517572206.genes.fna leifsona_2522572063.genes.fna pseud_2228664007.genes.fna WCS365_cds.fna Col0.fasta > notflavo_blastdb.fasta

# cat agrobacterium_2561511224.genes.fna bacillus_2517572206.genes.fna flavobacterium_2563366720.genes.fna pseud_2228664007.genes.fna WCS365_cds.fna Col0.fasta > notleif_blastdb.fasta

# cat agrobacterium_2561511224.genes.fna bacillus_2517572206.genes.fna flavobacterium_2563366720.genes.fna leifsona_2522572063.genes.fna WCS365_cds.fna Col0.fasta > notpseud_blastdb.fasta

# cat agrobacterium_2561511224.genes.fna bacillus_2517572206.genes.fna flavobacterium_2563366720.genes.fna leifsona_2522572063.genes.fna pseud_2228664007.genes.fna  Col0.fasta > notWCS365_blastdb.fasta

# # make blastdbs

# makeblastdb -in all_blastdb.fasta -dbtype nucl -out all_blastdb
# makeblastdb -in notagro_blastdb.fasta -dbtype nucl -out notagro_blastdb
# makeblastdb -in notbaci_blastdb.fasta -dbtype nucl -out notbaci_blastdb
# makeblastdb -in notflavo_blastdb.fasta -dbtype nucl -out notflavo_blastdb
# makeblastdb -in notleif_blastdb.fasta -dbtype nucl -out notleif_blastdb
# makeblastdb -in notpseud_blastdb.fasta -dbtype nucl -out notpseud_blastdb
# makeblastdb -in notWCS365_blastdb.fasta -dbtype nucl -out notWCS365_blastdb


In [ ]:
# #!/bin/sh

# #run blastdb

# blastn -query agro_possible.fasta  -db blast_dbs/notagro_blastdb -evalue 0.001 -num_threads 4 -outfmt 10 -out agro_blast.csv

# blastn -query baci_possible.fasta  -db blast_dbs/notbaci_blastdb -evalue 0.001 -num_threads 4 -outfmt 10 -out baci_blast.csv

# blastn -query flavo_possible.fasta  -db blast_dbs/notflavo_blastdb -evalue 0.001 -num_threads 4 -outfmt 10 -out flavo_blast.csv

# blastn -query leif_possible.fasta  -db blast_dbs/notleif_blastdb -evalue 0.001 -num_threads 4 -outfmt 10 -out leif_blast.csv

# blastn -query pseud_possible.fasta  -db blast_dbs/notpseud_blastdb -evalue 0.001 -num_threads 4 -outfmt 10 -out pseud_blast.csv

# blastn -query WCS365_possible.fasta  -db blast_dbs/notWCS365_blastdb -evalue 0.001 -num_threads 4 -outfmt 10 -out WCS365_blast.csv

### Post Blast Processing

In [ ]:

baci21_blast_hits = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/baci21_blast.csv"
baci21_annotation_filepath = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/syncom5/BacspUNC41MFS5_2/IMG_Data/2563366514/2563366514.ko.tab.txt"

bacillus21_output = post_blast_processing(baci21_blast_hits,baci21_probes,baci21_annotation_filepath)

bacillus21_good = bacillus21_output[0]
bacillus21_KO = bacillus21_output[1]

In [6]:
# Agrobacterium

agro_blast_hits = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/agro_blast.csv"
agro_annotation_filepath = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/Agrsp33MFTa11/IMG_Data/2561511224/2561511224.ko.tab.txt"

agro_output = post_blast_processing(agro_blast_hits,agro_probes,agro_annotation_filepath)

agro_good = agro_output[0]
agro_KO = agro_output[1]

# Bacillus

baci_blast_hits = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/baci_blast.csv"
baci_annotation_filepath = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/Bacillussp105MF/IMG_Data/2517572206/2517572206.ko.tab.txt"

bacillus_output = post_blast_processing(baci_blast_hits,baci_probes,baci_annotation_filepath)

bacillus_good = bacillus_output[0]
bacillus_KO = bacillus_output[1]

# Flavobacterium

flavo_blast_hits = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/flavo_blast.csv"
flavo_annotation_filepath = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/Flasp40S8/IMG_Data/2563366720/2563366720.ko.tab.txt"

flavo_output = post_blast_processing(flavo_blast_hits,flavo_probes,flavo_annotation_filepath)

flavo_good = flavo_output[0]
flavo_KO = flavo_output[1]

# Flavobacterium

leif_blast_hits = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/leif_blast.csv"
leif_annotation_filepath = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/Leifsoniasp109/IMG_Data/2522572063/2522572063.ko.tab.txt"

leif_output = post_blast_processing(leif_blast_hits,leif_probes,leif_annotation_filepath)

leif_good = leif_output[0]
leif_KO = leif_output[1]

# Pseudomonas

pseud_blast_hits = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/pseud_blast.csv"
pseud_annotation_filepath = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/PsespKD5_2/IMG_Data/2228664007/2228664007.ko.tab.txt"

pseud_output = post_blast_processing(pseud_blast_hits,pseud_probes,pseud_annotation_filepath)

pseud_good = pseud_output[0]
pseud_KO = pseud_output[1]

# # WCS365 - doesn't have KO annotation file as I didn't download it from JGI. However, the fasta names have the gene names, so I just need to filter it a bit.

wcs365_blast_hits = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/wcs365_blast.csv"
wcs365_annotation_filepath = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/genomes/Flasp40S8/IMG_Data/2563366720/2563366720.ko.tab.txt"

# wcs365_output = post_blast_processing(wcs365_blast_hits,wcs365_probes,wcs365_annotation_filepath)

# wcs365_good = wcs365_output[0]
# wcs365_KO = wcs365_output[1]

# WCS365
As this is a GenBank file, not a JGI file, it's a slightly different format. The below code cells work in much the same way as the initial functions, just modified slightly in respect to the different format. Instead of referencing the KO files (which aren't available from GenBank), I can split up the Fasta ID to extract the gene annotation.

In [7]:
possible_probes = wcs365_probes 
path_to_blast_hits = wcs365_blast_hits

In [8]:
#import blast hits and add column names
blast_hits = pd.read_csv(path_to_blast_hits, names = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"], usecols=["qseqid"])
#use it to filter our possible_probes
probes_df = pd.DataFrame(possible_probes, columns = ["probe_seq", "start_position_in_gene", "melting temp", "GC", "geneID" ]) #relabelling
probes_df['ID'] = probes_df['start_position_in_gene'].astype(str) + '_of_' + probes_df['geneID'].astype(str)
unique_probes = probes_df.loc[~probes_df['ID'].isin(blast_hits['qseqid'])].reset_index(drop=True)

In [9]:
#split 52-mer probes into 25-mer probes and recheck melting temperature and gc content

unique_probes['probe1'] = unique_probes.loc[:,'probe_seq'].str[0:25]
unique_probes['probe2'] = unique_probes.loc[:,'probe_seq'].str[27:50] # 2 nt gap.
#recheck melting temp
probe1_series = pd.Series(unique_probes.loc[:,'probe1'])
probe2_series = pd.Series(unique_probes.loc[:,'probe2'])
mt_list1 = []
mt_list2 = []

for probe in probe1_series:
    melting_temp = mt.Tm_NN(probe,dnac1=250, dnac2=250, Na=50)
    mt_list1.append(melting_temp)

for probe in probe2_series:
    melting_temp = mt.Tm_NN(probe,dnac1=250, dnac2=250, Na=50)
    mt_list2.append(melting_temp)

unique_probes['mt1'] = mt_list1
unique_probes['mt2'] = mt_list2

In [10]:

# and gc content of these half probes
gc_list1 = []
gc_list2 = []

for probe in probe1_series:
    gc_content = gc_fraction(probe)
    gc_list1.append(gc_content)

for probe in probe2_series:
    gc_content = gc_fraction(probe)
    gc_list2.append(gc_content)

unique_probes['gc1'] = gc_list1
unique_probes['gc2'] = gc_list2

In [11]:
#filter to useful probes
good_probes = unique_probes[
    (abs(unique_probes['mt1'] - 60) <= 2) &
    (abs(unique_probes['mt2'] - 60) <= 2) &
    (0.4 <= unique_probes['gc1']) & (unique_probes['gc1'] <= 0.6) &
    (0.4 <= unique_probes['gc2']) & (unique_probes['gc2'] <= 0.6)           
    ]

In [ ]:
good_probes

In [12]:
#find how many probes per gene and attach KO annotations
probes_by_geneID = pd.DataFrame(good_probes['geneID'].value_counts())
probes_by_geneID = probes_by_geneID.rename(columns={"count": "probe_count"})


In [13]:
#this is a bit messy, but it threw up a "Columns must be same length as key" error when I tried to do it on one line. This may possibly indicate some of these strings aren't splitting correctly... oh well!

wcs365_annotated = probes_by_geneID
wcs365_annotated['fasta_ID'] = wcs365_annotated.index
wcs365_annotated['locus_tag'] = wcs365_annotated.fasta_ID.str.split('[',expand=True)[1]
wcs365_annotated['protein_name'] = wcs365_annotated.fasta_ID.str.split('[',expand=True)[2]
wcs365_annotated['protein_id'] = wcs365_annotated.fasta_ID.str.split('[',expand=True)[3]
wcs365_annotated['locus'] = wcs365_annotated.fasta_ID.str.split('[',expand=True)[4]
wcs365_annotated['gb_key'] = wcs365_annotated.fasta_ID.str.split('[',expand=True)[5]


In [8]:
bacillus_KO.to_csv('bacillus21_gene_hits.csv', index =False)

# Output Possible Probes

In [ ]:
#output to csv
agro_KO.to_csv('agro_gene_hazel.csv', index=False)
bacillus_KO.to_csv('bacillus_gene_hits.csv', index =False)
bacillus21_KO.to_csv('bacillus_gene_hits.csv', index =False)
flavo_KO.to_csv('flavo_gene_hits.csv', index =False)
leif_KO.to_csv('leif_gene_hits.csv', index =False)
pseud_KO.to_csv('pseud_gene_hits.csv', index =False)
wcs365_annotated.to_csv('wcs365_gene_hits.csv', index =False)

Siyu has reviewed the above cvs files and picked out some genes of interest. I'm going to filter our good probes to those genes to then be able to pick out a selection. I'll then double check with another BLAST, and send some to Tatsuya for a review!

In [9]:
baci21_hk = bacillus_good[(bacillus_good['geneID'] == '2563431447') | (bacillus_good['geneID'] == '2563430930') | (bacillus_good['geneID'] == '2563428301') | (bacillus_good['geneID'] == '2563432503')] 

In [ ]:
#hk as in housekeeping
agro_hk = agro_good[(agro_good['geneID'] == '2562541429') | (agro_good['geneID'] == '2562541430') | (agro_good['geneID'] == '2230707588') | (agro_good['geneID'] == '2562541423')] 
baci_hk = bacillus_good[(bacillus_good['geneID'] == '2518125829') | (bacillus_good['geneID'] == '2518121137') | (bacillus_good['geneID'] == '2518125439') | (bacillus_good['geneID'] == '2518122162')] 
flavo_hk = flavo_good[(flavo_good['geneID'] == '2563781565') | (flavo_good['geneID'] == '2563782279') | (flavo_good['geneID'] == '2563784830')] 
leif_hk = leif_good[(leif_good['geneID'] == '2522795509') | (leif_good['geneID'] == '2522794802') | (leif_good['geneID'] == '2522795262') | (leif_good['geneID'] == '2522794626') | (leif_good['geneID'] == '2522794847')] 
pseud_hk = pseud_good[(pseud_good['geneID'] == '2230707191') | (pseud_good['geneID'] == '2230707195') | (pseud_good['geneID'] == '2230707588') | (pseud_good['geneID'] == '2230710753')] 


wcs365_hk = good_probes[(good_probes['geneID'] == 'lcl|NZ_PHHS01000018.1_cds_WP_100264311.1_2000_[locus_tag=CVG87_RS26585]_[protein=elongation_factor_Tu]_[frame=3]_[protein_id=WP_100264311.1]_[location=<1..836]_[gbkey=CDS]') | (good_probes['geneID'] == 'lcl|NZ_PHHS01000003.1_cds_WP_100263780.1_3513_[locus_tag=CVG87_RS08380]_[protein=UvrD-helicase_domain-containing_protein]_[protein_id=WP_100263780.1]_[location=complement(302828..305299)]_[gbkey=CDS]') | (good_probes['geneID'] == 'lcl|NZ_PHHS01000016.1_cds_WP_003197704.1_1832_[locus_tag=CVG87_RS25720]_[protein=YhbY_family_RNA-binding_protein]_[protein_id=WP_003197704.1]_[location=57643..57951]_[gbkey=CDS]') | (good_probes['geneID'] == 'lcl|NZ_PHHS01000002.1_cds_WP_043042072.1_2310_[locus_tag=CVG87_RS04470]_[protein=DNA_topoisomerase_III]_[protein_id=WP_043042072.1]_[location=127667..129613]_[gbkey=CDS]')] 



In [ ]:
#save to fasta files note! that this will append onto existing files
baci21_hk_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/baci21_hk.fasta", "a")
for hk_probe in baci21_hk.iterrows():
    string = f">{hk_probe[1][5]} \n" + str(hk_probe[1][0]) + "\n"
    baci21_hk_fasta.write(string)
baci21_hk_fasta.close()


/var/folders/d1/t36jr2g51dl5124ky8htph_r0000gq/T/ipykernel_19144/4289314936.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  string = f">{hk_probe[1][5]} \n" + str(hk_probe[1][0]) + "\n"


In [52]:
#save to fasta files note! that this will append onto existing files
agro_hk_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/agro_hk.fasta", "a")
for hk_probe in agro_hk.iterrows():
    string = f">{hk_probe[1][5]} \n" + str(hk_probe[1][0]) + "\n"
    agro_hk_fasta.write(string)
agro_hk_fasta.close()

baci_hk_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/baci_hk.fasta", "a")
for hk_probe in baci_hk.iterrows():
    string = f">{hk_probe[1][5]} \n" + str(hk_probe[1][0]) + "\n"
    baci_hk_fasta.write(string)
baci_hk_fasta.close()

flavo_hk_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/flavo_hk.fasta", "a")
for hk_probe in flavo_hk.iterrows():
    string = f">{hk_probe[1][5]} \n" + str(hk_probe[1][0]) + "\n"
    flavo_hk_fasta.write(string)
flavo_hk_fasta.close()

leif_hk_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/leif_hk.fasta", "a")
for hk_probe in leif_hk.iterrows():
    string = f">{hk_probe[1][5]} \n" + str(hk_probe[1][0]) + "\n"
    leif_hk_fasta.write(string)
leif_hk_fasta.close()

pseud_hk_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/pseud_hk.fasta", "a")
for hk_probe in pseud_hk.iterrows():
    string = f">{hk_probe[1][5]} \n" + str(hk_probe[1][0]) + "\n"
    pseud_hk_fasta.write(string)
pseud_hk_fasta.close()

wcs365_hk_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/wcs365_hk.fasta", "a")
for hk_probe in wcs365_hk.iterrows():
    string = f">{hk_probe[1][5]} \n" + str(hk_probe[1][0]) + "\n"
    wcs365_hk_fasta.write(string)
wcs365_hk_fasta.close()

/var/folders/d1/t36jr2g51dl5124ky8htph_r0000gq/T/ipykernel_6316/2492245522.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  string = f">{hk_probe[1][5]} \n" + str(hk_probe[1][0]) + "\n"
/var/folders/d1/t36jr2g51dl5124ky8htph_r0000gq/T/ipykernel_6316/2492245522.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  string = f">{hk_probe[1][5]} \n" + str(hk_probe[1][0]) + "\n"
/var/folders/d1/t36jr2g51dl5124ky8htph_r0000gq/T/ipykernel_6316/2492245522.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with Dat

ran the above through blast against a blastdb of all the other protein coding genomes in the syncom as so:
(with the exception of genomes Rhodococcus_erythropolis_339MFSha3.1_2643221496, Ochrobactrum_sp._370MFChir3.1_2643221500, and Variovorax_paradoxus_CL14_2643221508, which for whatever reason were unavailable on JGI. Instead, I used Rhodococcus erythropolis 2904535858, Ochrobactrum sp. 2824536489, and Variovorax paradoxus 4MFCol3.1 2517572245 respectively.)

blastn -query wcs365_hk.fasta  -db blast_dbs/not5syncom_blastdb -evalue 0.0001 -num_threads 4 -outfmt 10 -out wcs365_hk_blast.csv


In [ ]:
agro_hk_hits = pd.read_csv("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/agro_hk_blast.csv", names = ["probe_id", "hit_id","pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"])
baci_hk_hits = pd.read_csv("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/baci_hk_blast.csv", names = ["probe_id", "hit_id","pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"])
flavo_hk_hits = pd.read_csv("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/flavo_hk_blast.csv", names = ["probe_id", "hit_id","pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"])
leif_hk_hits = pd.read_csv("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/leif_hk_blast.csv", names = ["probe_id", "hit_id","pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"])
pseud_hk_hits = pd.read_csv("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/pseud_hk_blast.csv", names = ["probe_id", "hit_id","pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"])
wcs365_hk_hits = pd.read_csv("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/wcs365_hk_blast.csv", names = ["probe_id", "hit_id","pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"])


In [ ]:
baci21_hk_hits = pd.read_csv("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/baci21_hk_blast.csv", names = ["probe_id", "hit_id","pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"])

In [ ]:
baci21_unique_hk = baci21_hk.loc[~baci21_hk['ID'].isin(baci21_hk_hits['probe_id'])].reset_index(drop=True)
baci21_final_probes = pd.merge(baci21_unique_hk, bacillus21_KO, left_on='geneID', right_on='gene_oid').iloc[:,[4,6,7,8,9,10,11,16,17]]

In [ ]:
agro_unique_hk = agro_hk.loc[~agro_hk['ID'].isin(agro_hk_hits['probe_id'])].reset_index(drop=True)
agro_final_probes = pd.merge(agro_unique_hk, agro_KO, left_on='geneID', right_on='gene_oid').iloc[:,[4,6,7,8,9,10,11,16,17]]

baci_unique_hk = baci_hk.loc[~baci_hk['ID'].isin(baci_hk_hits['probe_id'])].reset_index(drop=True)
baci_final_probes = pd.merge(baci_unique_hk, bacillus_KO, left_on='geneID', right_on='gene_oid').iloc[:,[4,6,7,8,9,10,11,16,17]]

flavo_unique_hk = flavo_hk.loc[~flavo_hk['ID'].isin(flavo_hk_hits['probe_id'])].reset_index(drop=True)
flavo_final_probes = pd.merge(flavo_unique_hk, flavo_KO, left_on='geneID', right_on='gene_oid').iloc[:,[4,6,7,8,9,10,11,16,17]]

leif_unique_hk = leif_hk.loc[~leif_hk['ID'].isin(leif_hk_hits['probe_id'])].reset_index(drop=True)
leif_final_probes = pd.merge(leif_unique_hk, leif_KO, left_on='geneID', right_on='gene_oid').iloc[:,[4,6,7,8,9,10,11,16,17]]

pseud_unique_hk = pseud_hk.loc[~pseud_hk['ID'].isin(pseud_hk_hits['probe_id'])].reset_index(drop=True)
pseud_final_probes = pd.merge(pseud_unique_hk, pseud_KO, left_on='geneID', right_on='gene_oid').iloc[:,[4,6,7,8,9,10,11,16,17]]

wcs365_unique_hk = wcs365_hk.loc[~wcs365_hk['ID'].isin(wcs365_hk_hits['probe_id'])].reset_index(drop=True)
wcs365_final_probes = pd.merge(wcs365_unique_hk, wcs365_annotated, left_on='geneID', right_on='geneID').iloc[:,[5,6,7,8,9,10,11]]

In [88]:
# output to csv
agro_final_probes.to_csv('agro_final_probes.csv', index=False)
baci_final_probes.to_csv('bacillus_final_probes.csv', index =False)
flavo_final_probes.to_csv('flavo_final_probes.csv', index =False)
leif_final_probes.to_csv('leif_final_probes.csv', index =False)
pseud_final_probes.to_csv('pseud_final_probes.csv', index =False)
wcs365_final_probes.to_csv('wcs365_final_probes.csv', index =False)


In [13]:
baci21_final_probes.to_csv('bacillus21_final_probes.csv', index =False)

reverse transcriptase-ing everything

In [50]:
to_be_rt_df = pd.read_csv("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/probes_output/final_output/rev_comped.csv")

probe_list = []
rt_list = []

for probe in to_be_rt_df['cds_sequence'].values:
    probe_list.append(Seq(probe))
for probe in probe_list:
    rt_list.append(probe.reverse_complement())

to_be_rt_df['rev_comp'] = rt_list
to_be_rt_df.to_csv('rev_comped.csv', index=False)

# Generating a "General Probe" from the 16S rRNA Gene

In [ ]:
fasta_filepath = "/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/all_bac_probe/16S_conserved.faa"
probes_16S = possible_probes(fasta_filepath)
probes_16S

In [28]:
c16S_fasta = open("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/c16S_possible.fasta", "a")
for maybe_probe in probes_16S:
    string = f">{maybe_probe[1]}_of_{maybe_probe[4]} \n" + str(maybe_probe[0]) + "\n"
    c16S_fasta.write(string)
c16S_fasta.close()


In [43]:
def c16S_post_blast_processing(blast_hits_filepath,possible_probes):
    #import blast hits and add column names
    blast_hits = pd.read_csv(blast_hits_filepath, names = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"], usecols=["qseqid"])
    #use it to filter our possible_probes
    probes_df = pd.DataFrame(possible_probes, columns = ["probe_seq", "start_position_in_gene", "melting temp", "GC", "geneID" ]) #relabelling
    probes_df['ID'] = probes_df['start_position_in_gene'].astype(str) + '_of_' + probes_df['geneID'].astype(str)
    unique_probes = probes_df.loc[~probes_df['ID'].isin(blast_hits['qseqid'])].reset_index(drop=True)

    #split 52-mer probes into 25-mer probes and recheck melting temperature and gc content

    unique_probes['probe1'] = unique_probes.loc[:,'probe_seq'].str[0:25]
    unique_probes['probe2'] = unique_probes.loc[:,'probe_seq'].str[27:50] # 2 nt gap.
    #recheck melting temp
    probe1_series = pd.Series(unique_probes.loc[:,'probe1'])
    probe2_series = pd.Series(unique_probes.loc[:,'probe2'])
    mt_list1 = []
    mt_list2 = []

    for probe in probe1_series:
        melting_temp = mt.Tm_NN(probe,dnac1=250, dnac2=250, Na=50)
        mt_list1.append(melting_temp)

    for probe in probe2_series:
        melting_temp = mt.Tm_NN(probe,dnac1=250, dnac2=250, Na=50)
        mt_list2.append(melting_temp)

    unique_probes['mt1'] = mt_list1
    unique_probes['mt2'] = mt_list2

    # and gc content of these half probes
    gc_list1 = []
    gc_list2 = []

    for probe in probe1_series:
        gc_content = gc_fraction(probe)
        gc_list1.append(gc_content)

    for probe in probe2_series:
        gc_content = gc_fraction(probe)
        gc_list2.append(gc_content)

    unique_probes['gc1'] = gc_list1
    unique_probes['gc2'] = gc_list2

    #filter to useful probes
    good_probes = unique_probes[
        (abs(unique_probes['mt1'] - 60) <= 2) &
        (abs(unique_probes['mt2'] - 60) <= 2) &
        (0.4 <= unique_probes['gc1']) & (unique_probes['gc1'] <= 0.6) &
        (0.4 <= unique_probes['gc2']) & (unique_probes['gc2'] <= 0.6)           
        ]
    return(good_probes)

In [45]:
c16S_unique = c16S_post_blast_processing("/Users/guf24vol/Documents/nobori_PHYTOMap_rotation/probes/c16S_blast.csv",probes_16S)
c16S_unique.to_csv('c16S_unique_final_probes.csv', index =False)

In [ ]:
c16S_unique